In [1]:
import os
os.chdir('../')

In [2]:
from extract import open_server_users
from extract import open_server_engage
from extract import open_server_payment
from extract import open_server_battle
from extract import open_server_resource
from extract import open_server_troop
from extract import server_open_nd
from extract import open_server_goldfarmer


from transform import gold_farmer_aggr

import numpy as np
from config import ETLConfig

### 1. Open server settings

1. Define the game you want to analyze
2. Define days after server open you want data from
3. Define server_ids

In [3]:
from camel_utils_x.camel_sql import SQLBase

In [4]:
game = 'aoz'

days = 10

server_ids = [123,234,5,6,7,8,9]

### 2. Define server open
Fields in returned dataframe will be used in making queries to each server

In [5]:
# unshifted version of server open
server_open_nd()

,open_time,open_date,open_10_date,open_10_date_id,open_date_id
server_id,,,,,
0,2020-08-10,2020-08-10,2020-08-20,20200820,20200810
1,2018-09-20,2018-09-20,2018-09-30,20180930,20180920
2,2018-10-25,2018-10-25,2018-11-04,20181104,20181025
3,2018-11-16,2018-11-16,2018-11-26,20181126,20181116
4,2018-11-18,2018-11-18,2018-11-28,20181128,20181118
...,...,...,...,...,...
351,2020-11-23,2020-11-23,2020-12-03,20201203,20201123
352,2020-11-26,2020-11-26,2020-12-06,20201206,20201126
353,2020-11-28,2020-11-28,2020-12-08,20201208,20201128


In [6]:
# shifted version of sever open
server_open_nd(offset = 10, days = 20)

,open_time,open_date,open_20_date,open_20_date_id,open_date_id
server_id,,,,,
0,2020-08-20,2020-08-20,2020-09-09,20200909,20200820
1,2018-09-30,2018-09-30,2018-10-20,20181020,20180930
2,2018-11-04,2018-11-04,2018-11-24,20181124,20181104
3,2018-11-26,2018-11-26,2018-12-16,20181216,20181126
4,2018-11-28,2018-11-28,2018-12-18,20181218,20181128
...,...,...,...,...,...
351,2020-12-03,2020-12-03,2020-12-23,20201223,20201203
352,2020-12-06,2020-12-06,2020-12-26,20201226,20201206
353,2020-12-08,2020-12-08,2020-12-28,20201228,20201208


### Making queries

*On User*

In [7]:
user = open_server_users(server_ids = [234],days = 5, offset = 3, fields = ['id','create_time','language'])

UserType: 234: 100%|████████████████████████████████████████| 1/1 [00:03<00:00,  3.78s/it]


**From this line, we see that server 234 is opened on 2020-5-3**

In [8]:
server_open_nd(game).loc[234]

open_time          2020-05-03 00:00:00
open_date                   2020-05-03
open_10_date                2020-05-13
open_10_date_id               20200513
open_date_id                  20200503
Name: 234, dtype: object

**Since we shifted 3 days, the start date wound be 2020-5-6**         
**Setting days = 5, enables stopping at 2020-5-11 (excluded)**

In [9]:
server_open_nd(game, offset = 3, days = 5).loc[234]

open_time         2020-05-06 00:00:00
open_date                  2020-05-06
open_5_date                2020-05-11
open_5_date_id               20200511
open_date_id                 20200506
Name: 234, dtype: object

**Check dates in dataframe**

In [10]:
sorted(user.date.dt.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*On engage / with same days and offset configurations*

In [11]:
engage = open_server_engage(server_ids = [234], days = 5, offset = 3)

Engagement 234: 100%|███████████████████████████████████████| 1/1 [00:01<00:00,  1.12s/it]


In [12]:
sorted(engage.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

**If a server open dataframe is  passed into query functions, no additional open server queries execution inside queryies**

### Define openserver task


In [13]:
from config import ETLConfig

In [14]:
task = dict(game = 'aoz',
            server_ids = [234,235,236],
            offset = 1,
            days = 5)

task['server_open'] = server_open_nd(**task)


user = open_server_users(**task, fields = ETLConfig.USER_INFO_FIELDS)

engage = open_server_engage(**task, fields = ETLConfig.ENGAGE_FIELDS)

payment = open_server_payment(**task, fields = ETLConfig.PAYMENT_FIELDS)

battle = open_server_battle(**task, fields = ETLConfig.BATTLE_FIELDS)

resource = open_server_resource(**task, fields = ETLConfig.RESOURCE_FIELDS)

troop = open_server_troop(**task,fields = ETLConfig.TROOP_FIELDS)

goldfarmer = open_server_goldfarmer(**task, fields = ETLConfig.GOLD_FARMER_FIELDS)

GoldFarmerRob 236: 100%|████████████████████████████████████| 3/3 [00:35<00:00, 11.83s/it]


*Check user table*

In [15]:
user.head(5)

,user_id,create_time,language,register_channel,login_channel,country_code,register_ip_country,bind_channel,device_id,device_model,source_channel,from_server,date,user_type,day_id
0,19741855,2020-05-04 06:59:18,13,camel_google,camel_google,KZ,SE,None,170289dc-e790-4f67-a29f-b0227b47d131,HUAWEI STK-LX1,None,234,2020-05-04,pure_roller,1
1,19750606,2020-05-04 04:24:57,3,camel_ios,camel_ios,NG,NG,None,A3B6629C-6FF7-47D1-AF29-3F33D0269F78,"iPhone8,1",None,234,2020-05-04,cur_init_user,1
2,19750734,2020-05-04 03:48:12,3,camel_ios,camel_ios,US,US,None,6B1FA7AB-BD6D-423F-94D1-535E708074F5,"iPhone10,1",None,234,2020-05-04,cur_init_user,1
3,19750736,2020-05-04 15:26:21,3,camel_ios,camel_ios,US,US,None,A3C03D6C-8E35-4D7F-8D39-B7280A20B657,"iPad7,6",None,234,2020-05-04,cur_init_user,1
4,19750852,2020-05-04 16:56:34,7,camel_ios,camel_ios,SA,SA,None,9799A3D6-AFA9-4C9B-A526-D77909D6CAF8,"iPhone12,5",None,234,2020-05-04,cur_init_user,1


In [16]:
user[['date','day_id','from_server']].drop_duplicates().sort_values(by = ['from_server','day_id'])

,date,day_id,from_server
0,2020-05-04,1,234
48907,2020-05-05,2,234
48518,2020-05-06,3,234
63476,2020-05-07,4,234
63385,2020-05-08,5,234
63611,2020-05-06,1,235
100583,2020-05-07,2,235
100872,2020-05-08,3,235
101040,2020-05-09,4,235
101173,2020-05-10,5,235


In [17]:
troop[['date','day_id','from_server']].drop_duplicates().sort_values(by = ['from_server','day_id'])

,date,day_id,from_server
0,2020-05-04,1,234
51430,2020-05-05,2,234
78093,2020-05-06,3,234
88925,2020-05-07,4,234
100194,2020-05-08,5,234
110715,2020-05-06,1,235
156569,2020-05-07,2,235
169470,2020-05-08,3,235
180154,2020-05-09,4,235
190634,2020-05-10,5,235


------------
------------

In [21]:
goldfarmer = open_server_goldfarmer(server_ids = [234], 
                                    server_open = server_open, 
                                    days = 5, 
                                    offset = 3,
                                   fields = ETLConfig.GOLD_FARMER_FIELDS)

gold_farmer_user_daily = gold_farmer_aggr(goldfarmer, ETLConfig)  

GoldFarmerRob 234: 100%|████████████████████████████████████| 1/1 [00:15<00:00, 15.69s/it]


In [22]:
gold_farmer_user_daily[(gold_farmer_user_daily.atk_count >0) & (gold_farmer_user_daily.def_count >0)]

,user_id,date,ip,name,device_id,user_lvl,city_lvl,alli_id,alli_full,alli_short,...,atk_death,atk_wound,atk_kill,def_online,def_survive,def_death,def_wound,def_kill,atk_count,def_count
21,19656035,2020-05-08,170.246.228.51,Commander19656035,6d4c2d2f-4971-47a0-a78e-9678ba1d3dd2,14.0,10.0,201824.0,novomundo,nm2,...,144.0,16.0,1824.0,2.0,0.0,0.0,172.0,6.0,2.0,8.0
100,19691963,2020-05-10,171.225.167.130,_TiNa...,f423f55f-c946-4413-b5e7-37d007fe8133,18.0,13.0,201195.0,Anhwmcietnam,Aev,...,1892.0,138.0,50316.0,0.0,0.0,0.0,0.0,0.0,52.0,188.0
102,19692246,2020-05-07,113.139.190.78,红眼兔子,eef10005-cbdf-494f-bc3d-681cc68721b4,16.0,9.0,0.0,,,...,0.0,0.0,0.0,0.0,0.0,0.0,72.0,6.0,4.0,2.0
103,19692246,2020-05-08,113.139.195.106,红眼兔子,eef10005-cbdf-494f-bc3d-681cc68721b4,17.0,11.0,201594.0,ASGARD,AGR,...,3938.0,336.0,15994.0,0.0,0.0,0.0,0.0,0.0,10.0,2.0
120,19694682,2020-05-09,144.134.150.147,DaggyDadII,C8EBE62D-3FCB-40B0-B183-2F38D3BB5EE0,18.0,13.0,201194.0,BloodNation3,BN3,...,4080.0,330.0,61504.0,2.0,0.0,0.0,0.0,0.0,24.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,19930526,2020-05-09,213.24.135.254,Марадёр@,81095796-dc91-49ff-a995-01d686542768,15.0,10.0,201594.0,ASGARD,AGR,...,2318.0,158.0,24944.0,8.0,6502.0,0.0,17486.0,3478.0,50.0,8.0
4452,19931498,2020-05-10,177.5.168.91,GABRIELR,d33374a1-0dc4-4456-83ae-a7aeb2ed769c,14.0,8.0,201824.0,novomundo,nm2,...,84.0,4.0,712.0,0.0,13900.0,0.0,29116.0,4258.0,2.0,4.0
4465,19985407,2020-05-10,117.111.11.63,철옹특급,3011db53-7399-4555-96c3-e98c829be2eb,11.0,8.0,205189.0,도시정벌,zkt,...,1272.0,96.0,8504.0,0.0,0.0,0.0,434.0,2.0,14.0,2.0
4475,20067419,2020-05-10,176.52.115.219,ferm_ds,dd24e0b4-82a4-4eb8-8ac6-5306bf0f074f,13.0,9.0,206194.0,RedRose2,RR2,...,0.0,0.0,0.0,2.0,5320.0,0.0,18604.0,1304.0,102.0,16.0
